In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import pandas as pd
import time
from IPython.display import display 

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

In [3]:
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn import tree

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.ensemble import RandomForestClassifier,  AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

In [5]:
#test
data = pd.read_pickle(r'data\subject101.pkl')

In [6]:
#train loso
data1 = pd.read_pickle(r'data\data_loso1.pkl')

In [7]:
#train cross-val
data2 = pd.read_pickle(r'data\subject106.pkl')

In [8]:
activities = data.groupby('activity')
activities1 = data1.groupby('activity')
activities2 = data2.groupby('activity')

Cut off first and last 1000 items, because activity starts end ends



In [9]:
acts = data.activity.unique()
#acts =  ['standing', 'walking','running','cycling']
data = data.loc[data['activity'].isin(acts)].copy()
for i in range(len(acts)):
    data.drop(data[data.activity==acts[i]].iloc[:1000].index, inplace=True)
    data.drop(data[data.activity==acts[i]].iloc[-1000:].index, inplace=True)


In [10]:
data1 = data1.loc[data1['activity'].isin(acts)].copy()
for y in range(len(acts)):
    data1.drop(data1[data1.activity==acts[y]].iloc[:1000].index, inplace=True)
    data1.drop(data1[data1.activity==acts[y]].iloc[-1000:].index, inplace=True)


In [11]:
data2 = data2.loc[data2['activity'].isin(acts)].copy()
for z in range(len(acts)):
    data2.drop(data2[data2.activity==acts[z]].iloc[:1000].index, inplace=True)
    data2.drop(data2[data2.activity==acts[z]].iloc[-1000:].index, inplace=True)


data.drop(data.index[(data.activityID!=3) & (data.activityID!=4) & (data.activityID!=5) & (data.activityID!=6)], inplace=True)


data.drop(data[data.activityID==3].iloc[:1000].index, inplace=True)
data.drop(data[data.activityID==3].iloc[-1000:].index, inplace=True)
data.drop(data[data.activityID==4].iloc[:1000].index, inplace=True)
data.drop(data[data.activityID==4].iloc[-1000:].index, inplace=True)
data.drop(data[data.activityID==5].iloc[:1000].index, inplace=True)
data.drop(data[data.activityID==5].iloc[-1000:].index, inplace=True)
data.drop(data[data.activityID==6].iloc[:1000].index, inplace=True)
data.drop(data[data.activityID==6].iloc[-1000:].index, inplace=True)

# Features

## Absolute Acceleration


absolute acceleration: $|a|=\sqrt{a_x^2 + a_y^2 + a_z^2}$

to get rid of the orientation of the device

In [12]:
def absacc(row):
    return np.sqrt(row['IMU_chest_ax1']**2 + row['IMU_chest_ay1']**2 + row['IMU_chest_az1']**2)/9.806

In [13]:
data['absacc'] = data.apply(absacc, axis=1)

In [14]:
data1['absacc'] = data1.apply(absacc, axis=1)

In [15]:
data2['absacc'] = data2.apply(absacc, axis=1)

## Max-Min Difference of absolute Acceleration

In [16]:
ws=512
dt = 1.0/100.0 # the activities were with 50Hz


In [17]:
data['accmax'] =  data['absacc'].rolling(window=ws,center=False).max() 
data['accmin'] = data['absacc'].rolling(window=ws,center=False).min() 

data['accmaxmindiff'] = data.accmax - data.accmin

In [18]:
data1['accmax'] =  data1['absacc'].rolling(window=ws,center=False).max() 
data1['accmin'] = data1['absacc'].rolling(window=ws,center=False).min() 

data1['accmaxmindiff'] = data1.accmax - data1.accmin

In [19]:
data2['accmax'] =  data2['absacc'].rolling(window=ws,center=False).max() 
data2['accmin'] = data2['absacc'].rolling(window=ws,center=False).min() 

data2['accmaxmindiff'] = data2.accmax - data2.accmin

## Fourier Transform of Rotation Rates


In [20]:
def fft_amplitude(s, kind='peak'):
    
    # don't forget the windowing to get rid of the leakage effect
    hann = np.hanning(len(s)) 
    
    # do the FFT with Hanning Window
    Yhann = np.fft.fft(hann*s)
    
    N = int(len(Yhann)/2+1)
    Y = 2*(np.abs(Yhann[:N])/N) # right half is enough info(positive freqs only)
    
    # frequency axis, if needed
    fa = 1.0/dt
    f = np.linspace(0, fa/2.0, N, endpoint=True)
    
    if kind=='peak':
        return np.max(Y) # just return the maximum peak amplitude
    elif kind=='periodicity':
        return np.max(Y) / np.mean(Y) # return periodicity
    elif kind=='full':
        return f, Y # return the full spectrum

In [21]:
data['fftamppeak'] = data['IMU_chest_rotz'].rolling(window=1*ws,center=False).apply(fft_amplitude, raw=False)


In [22]:
data1['fftamppeak'] = data1['IMU_chest_rotz'].rolling(window=1*ws,center=False).apply(fft_amplitude, raw=False)


In [23]:
data2['fftamppeak'] = data2['IMU_chest_rotz'].rolling(window=1*ws,center=False).apply(fft_amplitude, raw=False)


Because of the rolling_ functions, there is overlap between the activity features and the labels, corresponding to it. We have to delete some rows (length of window), before using a classifier.



In [24]:
for ii in range(len(acts)):
    data.drop(data[data.activity==acts[ii]].iloc[0:int(ws)-1].index, inplace=True)

In [25]:
for iii in range(len(acts)):
    data1.drop(data1[data1.activity==acts[iii]].iloc[0:int(ws)-1].index, inplace=True)

In [26]:
for iiii in range(len(acts)):
    data2.drop(data2[data2.activity==acts[iiii]].iloc[0:int(ws)-1].index, inplace=True)

data.drop(data[data.activityID==3].iloc[0:int(ws)-1].index, inplace=True)
data.drop(data[data.activityID==4].iloc[0:int(ws)-1].index, inplace=True)
data.drop(data[data.activityID==5].iloc[0:int(ws)-1].index, inplace=True)
data.drop(data[data.activityID==6].iloc[0:int(ws)-1].index, inplace=True)

In [27]:
#data.to_pickle(r'data\data1_feats.pkl')

# Classification

In [28]:
def get_train_test(df, y_col, x_cols, ratio):
    """ 
    This method transforms a dataframe into a train and test set, for this you need to specify:
    1. the ratio train : test (usually 0.7)
    2. the column with the Y_values
    """
#    mask = np.random.rand(len(df)) < ratio
#    df_train = df[mask]
#    df_test = df[~mask]
    
    df_train=df.sample(frac=ratio,random_state=200)
    df_test=df.drop(df_train.index)
       
    Y_train = df_train[y_col].values
    Y_test = df_test[y_col].values
    X_train = df_train[x_cols].values
    X_test = df_test[x_cols].values
    return df_train, df_test, X_train, Y_train, X_test, Y_test

Each ensemble algorithm is demonstrated using 10 fold cross validation, a standard technique used to estimate the performance of any machine learning algorithm on unseen data.

In [29]:
seed = 7
num_trees = 100
max_features = 2

In [30]:
dict_classifiers = {
    #"Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(n_neighbors =3),
    #"Linear SVM": SVC(kernel='rbf', C=1.0, gamma=0.5),
    "Decision Tree": tree.DecisionTreeClassifier(),
    #"Random Forest10": RandomForestClassifier(n_estimators=10),
    #"Neural Net": MLPClassifier(alpha = 1),
    #"Naive Bayes": GaussianNB(),
    
    #Bagging Algos
    "Bagged Decision Trees": BaggingClassifier(base_estimator=tree.DecisionTreeClassifier(), n_estimators=num_trees, random_state=seed),
    "RF": RandomForestClassifier(n_estimators=num_trees, max_features=max_features),
    "Extra Trees": ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features),
    #Boosting Algos
    #"AdaBoost": AdaBoostClassifier(n_estimators=num_trees, random_state=seed),
    #Stochastic Gradient Boosting
    "Stochastic Gradient Boosting": GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
        
}

In [31]:
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    The trained models, and accuracies are saved in a dictionary. The reason to use a dictionary
    is because it is very easy to save the whole dictionary with the pickle module.
    
    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
    So it is best to train them on a smaller dataset first and 
    decide whether you want to comment them out or not based on the test accuracy score.
    """
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.process_time()
        classifier.fit(X_train, Y_train)
        t_end = time.process_time()
        t_diff = t_end - t_start
        
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        Y_true = classifier.predict(X_test)
        accuracy = accuracy_score(Y_true, Y_test)
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'train_time': t_diff, 'y_true': Y_true, 'accuracy_score': accuracy, 'y_pred': Y_test}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models

In [32]:
def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    accuracy_s = [dict_models[key]['accuracy_score'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),5)), columns = ['classifier', 'train_score', 'test_score', 'train_time','accuracy_score'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
        df_.loc[ii, 'accuracy_score'] = accuracy_s[ii]*100
    
    display(df_.sort_values(by=sort_by, ascending=False))

In [33]:
labels = data['activity'].values
np.shape(labels)

(217076,)

### Classification with Features

In [34]:
featurevector = ['accmaxmindiff','fftamppeak']

features = data[featurevector].values
np.shape(features)

(217076, 2)

In [35]:
#features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.4, random_state=42)

In [36]:
y_col = 'activity'

train_test_ratio = 0.7

#df_train, df_test, features_train, labels_train, features_test, labels_test = get_train_test(data, y_col, featurevector, train_test_ratio)

In [37]:
Y_train = data1[y_col].values
X_train = data1[featurevector].values

Y_test = data[y_col].values
X_test = data[featurevector].values

Y_train1 = data2[y_col].values
X_train1 = data2[featurevector].values


## LOSO

In [38]:
dict_models = batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 6)

trained Nearest Neighbors in 2.56 s
trained Decision Tree in 6.67 s
trained Bagged Decision Trees in 355.72 s
trained RF in 353.92 s
trained Extra Trees in 90.41 s
trained Stochastic Gradient Boosting in 1489.86 s


In [39]:
display_dict_models(dict_models)

,classifier,train_score,test_score,train_time,accuracy_score
5,Stochastic Gradient Boosting,0.769729,0.583938,1489.859375,58.393834
1,Decision Tree,0.999996,0.495499,6.671875,49.549927
3,RF,0.999984,0.495232,353.921875,49.523208
2,Bagged Decision Trees,0.999992,0.492698,355.718750,49.269841
4,Extra Trees,0.999998,0.487553,90.406250,48.755275
0,Nearest Neighbors,0.955330,0.473074,2.562500,47.307395


## Cross Validation

In [40]:
dict_models1 = batch_classify(X_train1, Y_train1, X_test, Y_test, no_classifiers = 6)
display_dict_models(dict_models1)

trained Nearest Neighbors in 0.53 s
trained Decision Tree in 0.81 s
trained Bagged Decision Trees in 36.86 s
trained RF in 37.22 s
trained Extra Trees in 8.30 s
trained Stochastic Gradient Boosting in 145.19 s


,classifier,train_score,test_score,train_time,accuracy_score
5,Stochastic Gradient Boosting,0.954111,0.429375,145.187500,42.937497
4,Extra Trees,1.000000,0.418301,8.296875,41.830050
3,RF,1.000000,0.414247,37.218750,41.424662
2,Bagged Decision Trees,0.999995,0.413708,36.859375,41.370764
1,Decision Tree,1.000000,0.413680,0.812500,41.368000
0,Nearest Neighbors,0.990745,0.407581,0.531250,40.758076


## Normal

In [41]:
df_train, df_test, features_train, labels_train, features_test, labels_test = get_train_test(data1, y_col, featurevector, train_test_ratio)

In [42]:
dict_models2 = batch_classify(features_train, labels_train, features_test, labels_test, no_classifiers = 6)
display_dict_models(dict_models2)

trained Nearest Neighbors in 2.66 s
trained Decision Tree in 5.64 s
trained Bagged Decision Trees in 371.83 s
trained RF in 375.77 s
trained Extra Trees in 120.28 s
trained Stochastic Gradient Boosting in 1526.81 s


,classifier,train_score,test_score,train_time,accuracy_score
4,Extra Trees,0.999999,0.995253,120.281250,99.525257
3,RF,0.999995,0.993896,375.765625,99.389616
2,Bagged Decision Trees,0.999994,0.993745,371.828125,99.374545
1,Decision Tree,0.999999,0.992841,5.640625,99.284117
0,Nearest Neighbors,0.956087,0.948858,2.656250,94.885836
5,Stochastic Gradient Boosting,0.770700,0.789983,1526.812500,78.998267


In [43]:
import pickle

# Store data (serialize)
with open(r'data\loso_class.pickle', 'wb') as handle:
    pickle.dump(dict_models, handle)
    
# Store data (serialize)
with open(r'data\cross_class.pickle', 'wb') as handle1:
    pickle.dump(dict_models1, handle1)
    
# Store data (serialize)
with open(r'data\sub106_class.pickle', 'wb') as handle2:
    pickle.dump(dict_models2, handle2)